<a href="https://colab.research.google.com/github/joshtimmons/llm-demos/blob/main/chat_with_document/question_answer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# How to "Chat" with a document

This is the accompanying material for the 11/8 AI session. 

In this session we will learn about:
1. text embeddings
2. vector stores
3. document chunking
4. document retrieval based on a prompted question
5. converting a retrieved document into an "answer"

In [ ]:
!pip install langchain transformers sentence_transformers "chromadb<=0.4.15"

First we're going to do some basic model setup. I'm using the lightweight flan-alpaca-large model that is tuned for assistive generation. 

This model plays two roles in our QA scenario:
1. It will figure out what text in our documents contains content that answers the question
2. It will combine the information we collect into a single answer

This is a small model by LLM standards at 0.7B parameters, but it's good enough for the job :-) 

In [ ]:
# This code downloads the model from the HuggingFace model hub and loads it into a pipeline.

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline, AutoModelForCausalLM
from langchain.llms import HuggingFacePipeline

model_id = "declare-lab/flan-alpaca-large"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(model_id)
pipe = pipeline("text2text-generation", model=model, tokenizer=tokenizer, device="cuda")
local_llm = HuggingFacePipeline(pipeline=pipe)

Next we're going to start the process of vectorizing and storing the text.

I'm using the first chapter of "The Wizard of Oz" by Frank Baum for illustrative purposes.

In [ ]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.docstore.document import Document
from langchain.prompts import PromptTemplate
from langchain.indexes.vectorstore import VectorstoreIndexCreator
from langchain.embeddings import HuggingFaceEmbeddings

In [ ]:
woz = """
Dorothy lived in the midst of the great Kansas prairies, with Uncle
Henry, who was a farmer, and Aunt Em, who was the farmer’s wife. Their
house was small, for the lumber to build it had to be carried by wagon
many miles. There were four walls, a floor and a roof, which made one
room; and this room contained a rusty looking cookstove, a cupboard for
the dishes, a table, three or four chairs, and the beds. Uncle Henry
and Aunt Em had a big bed in one corner, and Dorothy a little bed in
another corner. There was no garret at all, and no cellar—except a
small hole dug in the ground, called a cyclone cellar, where the family
could go in case one of those great whirlwinds arose, mighty enough to
crush any building in its path. It was reached by a trap door in the
middle of the floor, from which a ladder led down into the small, dark
hole.

When Dorothy stood in the doorway and looked around, she could see
nothing but the great gray prairie on every side. Not a tree nor a
house broke the broad sweep of flat country that reached to the edge of
the sky in all directions. The sun had baked the plowed land into a
gray mass, with little cracks running through it. Even the grass was
not green, for the sun had burned the tops of the long blades until
they were the same gray color to be seen everywhere. Once the house had
been painted, but the sun blistered the paint and the rains washed it
away, and now the house was as dull and gray as everything else.

When Aunt Em came there to live she was a young, pretty wife. The sun
and wind had changed her, too. They had taken the sparkle from her eyes
and left them a sober gray; they had taken the red from her cheeks and
lips, and they were gray also. She was thin and gaunt, and never smiled
now. When Dorothy, who was an orphan, first came to her, Aunt Em had
been so startled by the child’s laughter that she would scream and
press her hand upon her heart whenever Dorothy’s merry voice reached
her ears; and she still looked at the little girl with wonder that she
could find anything to laugh at.

Uncle Henry never laughed. He worked hard from morning till night and
did not know what joy was. He was gray also, from his long beard to his
rough boots, and he looked stern and solemn, and rarely spoke.

It was Toto that made Dorothy laugh, and saved her from growing as gray
as her other surroundings. Toto was not gray; he was a little black
dog, with long silky hair and small black eyes that twinkled merrily on
either side of his funny, wee nose. Toto played all day long, and
Dorothy played with him, and loved him dearly.

Today, however, they were not playing. Uncle Henry sat upon the
doorstep and looked anxiously at the sky, which was even grayer than
usual. Dorothy stood in the door with Toto in her arms, and looked at
the sky too. Aunt Em was washing the dishes.

From the far north they heard a low wail of the wind, and Uncle Henry
and Dorothy could see where the long grass bowed in waves before the
coming storm. There now came a sharp whistling in the air from the
south, and as they turned their eyes that way they saw ripples in the
grass coming from that direction also.

Suddenly Uncle Henry stood up.

“There’s a cyclone coming, Em,” he called to his wife. “I’ll go look
after the stock.” Then he ran toward the sheds where the cows and
horses were kept.

Aunt Em dropped her work and came to the door. One glance told her of
the danger close at hand.

“Quick, Dorothy!” she screamed. “Run for the cellar!”

Toto jumped out of Dorothy’s arms and hid under the bed, and the girl
started to get him. Aunt Em, badly frightened, threw open the trap door
in the floor and climbed down the ladder into the small, dark hole.
Dorothy caught Toto at last and started to follow her aunt. When she
was halfway across the room there came a great shriek from the wind,
and the house shook so hard that she lost her footing and sat down
suddenly upon the floor.

Then a strange thing happened.

The house whirled around two or three times and rose slowly through the
air. Dorothy felt as if she were going up in a balloon.

The north and south winds met where the house stood, and made it the
exact center of the cyclone. In the middle of a cyclone the air is
generally still, but the great pressure of the wind on every side of
the house raised it up higher and higher, until it was at the very top
of the cyclone; and there it remained and was carried miles and miles
away as easily as you could carry a feather.

It was very dark, and the wind howled horribly around her, but Dorothy
found she was riding quite easily. After the first few whirls around,
and one other time when the house tipped badly, she felt as if she were
being rocked gently, like a baby in a cradle.

Toto did not like it. He ran about the room, now here, now there,
barking loudly; but Dorothy sat quite still on the floor and waited to
see what would happen.

Once Toto got too near the open trap door, and fell in; and at first
the little girl thought she had lost him. But soon she saw one of his
ears sticking up through the hole, for the strong pressure of the air
was keeping him up so that he could not fall. She crept to the hole,
caught Toto by the ear, and dragged him into the room again, afterward
closing the trap door so that no more accidents could happen.

Hour after hour passed away, and slowly Dorothy got over her fright;
but she felt quite lonely, and the wind shrieked so loudly all about
her that she nearly became deaf. At first she had wondered if she would
be dashed to pieces when the house fell again; but as the hours passed
and nothing terrible happened, she stopped worrying and resolved to
wait calmly and see what the future would bring. At last she crawled
over the swaying floor to her bed, and lay down upon it; and Toto
followed and lay down beside her.

In spite of the swaying of the house and the wailing of the wind,
Dorothy soon closed her eyes and fell fast asleep.
"""

In order to create embeddings of our text, we have to break it down. I'm splitting the text by sentence in the hope that it gives us reasonable sizes.

We're using langchain's CharacterTextSplitter here to do the splitting.

In [ ]:
text_splitter = CharacterTextSplitter(separator= ".", chunk_size=250, chunk_overlap=0)
all_splits = text_splitter.split_text(woz)

In [ ]:
# Let's look at the first split. Note that it split the input at a sentence boundary, but not at every sentence boundary. It does try to fill the chunk size.
all_splits[0]

In [ ]:
# What is the embedding generated from that chunk?
embeddings = HuggingFaceEmbeddings()

embeddings.embed_query(all_splits[0])

This is almost too easy because a lot is happening behind the scenes that is hidden from us. 

1. We're using the LangChain HuggingFaceEmbeddings class. It uses BAAI/bge-large-en-v1.5 to generate the embeddings by default. More info is at https://huggingface.co/BAAI/bge-large-en-v1.5
2. It uses chromadb to convert the text chunks into embeddings and load them into a local database


In [ ]:
docsearch = Chroma.from_texts(all_splits, embeddings, metadatas=[{"source": str(i)} for i in range(len(all_splits))]).as_retriever()

print(f"there are {len(all_splits)} documents")

And this is how we ask the document a question.

1. We load a QA chain using langchain. This is a simple "stuff" chain that just crams the documents into the buffer and asks the question

See https://python.langchain.com/docs/modules/chains/document/stuff

In [ ]:
from langchain.chains.question_answering import load_qa_chain

chain = load_qa_chain(local_llm, chain_type="stuff")

query = "Where is the farmhouse?"
docs = docsearch.get_relevant_documents(query)

for d in docs:
  print(d)

# ask the chain our questions and give it the most relevant documents
chain({"input_documents": docs, "question": query})

There are 4 document chain strategies.

1. Stuff. The simplest one. Shown above.
2. The Refine documents chain constructs a response by looping over the input documents and iteratively updating its answer.
3. The map reduce documents chain first applies an LLM chain to each document individually (the Map step), treating the chain output as a new document. It then passes all the new documents to a separate combine documents chain to get a single output (the Reduce step).
4. The map re-rank documents chain runs an initial prompt on each document, that not only tries to complete a task but also gives a score for how certain it is in its answer. The highest scoring response is returned.

I find that "refine" generally gives good quality answers and runs a little faster than "map reduce" and "map re-rank."

In [ ]:
refine_prompt_template = (
    "The original question is as follows: {question}\n"
    "We have provided an existing answer: {existing_answer}\n"
    "We have the opportunity to refine the existing answer"
    "(only if needed) with some more context below.\n"
    "------------\n"
    "{context_str}\n"
    "------------\n"
    "Given the new context, refine the original answer to better "
    "answer the question. If the context doesn't answer the question, say you don't know. Do not make up an answer."
)
refine_prompt = PromptTemplate(
    input_variables=["question", "existing_answer", "context_str"],
    template=refine_prompt_template,
)


initial_qa_template = (
    "Context information is below. \n"
    "---------------------\n"
    "{context_str}"
    "\n---------------------\n"
    "Given the context information and not prior knowledge, "
    "answer the question. Do not make up an answer if you do not know: {question}\n"
)
initial_qa_prompt = PromptTemplate(
    input_variables=["context_str", "question"], template=initial_qa_template
)
chain = load_qa_chain(local_llm, chain_type="refine", return_refine_steps=True,
                     question_prompt=initial_qa_prompt, refine_prompt=refine_prompt)



In [ ]:
query = "How many times did the house spin?"
docs = docsearch.get_relevant_documents(query)

for d in docs:
  print(d)

chain({"input_documents": docs, "question": query}, return_only_outputs=True)


In [ ]:
query = "Where was Dorothy's house?"
docs = docsearch.get_relevant_documents(query)

for d in docs:
  print(d)

chain({"input_documents": docs, "question": query}, return_only_outputs=True)


In [ ]:
query = "List the people who lived in the house"
docs = docsearch.get_relevant_documents(query)

for d in docs:
  print(d)

chain({"input_documents": docs, "question": query}, return_only_outputs=True)



In [ ]:
query = "How far was the house carried?"
docs = docsearch.get_relevant_documents(query)

for d in docs:
  print(d)

chain({"input_documents": docs, "question": query}, return_only_outputs=True)
